In [46]:
import numpy as np
dataset = np.loadtxt('./tictac_multi.txt')

In [47]:
np.random.shuffle(dataset)

In [48]:
X = dataset[:, :9]
Y = dataset[:,9:]

In [49]:
print(X.shape)
print(Y.shape)

(6551, 9)
(6551, 9)


In [50]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from numpy import arange
from numpy import argmax
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from numpy.linalg import inv

In [51]:
def linearREG(train_X, test_X, train_Y, test_Y):
    print(test_X.shape)
    print(test_Y.shape)
    print(train_X.shape)
    print(train_Y.shape)
    
    #creating the output matrix
    Y_prediction = np.empty((np.shape(test_Y)[0], np.shape(test_Y)[1]))
    
    #linear regression for each columns
    for i in range(9):
        Y = train_Y[:, i]
        theta = np.linalg.pinv(train_X.T @ train_X) @ train_X.T @ Y
        pred = test_X @ theta
        Y_prediction[:, i] = pred
    res = Y_prediction
#     X = np.c_[np.ones(train_X.shape[0]),train_X]
#     X_transpose = X.T # riju
    
#     #inv_X = np.linalg.pinv(X)
#     #theta = np.dot(inv_X,np.dot(np.transpose(X),train_Y))
#     theta = inv(X_transpose.dot(X)).dot(X_transpose).dot(train_Y) # riju
#     X_t = np.c_[np.ones(test_X.shape[0]),test_X] 
#     Y_prediction = np.dot(X_t,theta)
#     res = Y_prediction
    
    
    #one hot encoding
    res = (res == res.max(axis = 1)[:, None]).astype(int)
        
    res_cm = res
    clf = res
    print("######### Y ############")
    print(test_Y)
    print("%%%%%%%%%%% T_RES %%%%%%%%%%%%%%%%%%%")
    print(res)
    threshold_res = res
  
    print('Mean Absolute Error:', metrics.mean_absolute_error(test_Y, threshold_res))
    print('Mean Squared Error:', metrics.mean_squared_error(test_Y, threshold_res))
    mse = np.sqrt(metrics.mean_squared_error(test_Y, threshold_res))
    print('Root Mean Squared Error:', mse)
    
    accuracy = 100 - (mse * 100)
    print("Accuracy: " + str(accuracy))
#     print('Accuracy: {:.2f}'.format(accuracy))
#     print("Accuracy = " + str(accuracy/res.shape[1]))
    print("#### CONFUSION MATRIX ##########")
    rounded_labels = np.argmax(test_Y, axis = 1)
    cm = confusion_matrix(rounded_labels, res_cm.argmax(axis = 1))
    cm = cm / cm.sum(axis=1)[:, np.newaxis] # normalizing the confusion matrix
    print(cm)
    return accuracy, clf

In [52]:
from skmultilearn.model_selection import IterativeStratification
accuracy_max = 0.0
accuracy_min = 100.0
accuracy_avg = 0.0

best_clf = 0

k_cross_fold = 10

skf = IterativeStratification(n_splits = k_cross_fold)
for train_index, test_index in skf.split(X, Y):
    #print(str(train_index[0]) + " and " + str(train_index[-1])  + " and " + "shape = " + str(train_index.shape))
    #print(str(test_index[0]) + " and " + str(test_index[-1])   + " and " + "shape = " + str(test_index.shape))
    train_X, test_X = X[train_index], X[test_index]
    train_Y, test_Y = Y[train_index], Y[test_index]
    
    accuracy, clf = linearREG(train_X, test_X, train_Y, test_Y)
    #accuracy *= 100.0
    #print("A:" + str(accuracy))
    if(accuracy > accuracy_max):
        accuracy_max = accuracy
        best_clf = clf
    accuracy_min = min(accuracy_min, accuracy)
    accuracy_avg += accuracy
    #print(accuracy_avg)
    print("#######################")
    
print("Max accuracy = " + str(accuracy_max))
print("Min accuracy = " + str(accuracy_min))
print("Avg accuracy = " + str(accuracy_avg/k_cross_fold))

(634, 9)
(634, 9)
(5917, 9)
(5917, 9)
######### Y ############
[[1. 1. 0. ... 0. 0. 1.]
 [1. 0. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 1. 0.]]
%%%%%%%%%%% T_RES %%%%%%%%%%%%%%%%%%%
[[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]]
Mean Absolute Error: 0.28776726253066953
Mean Squared Error: 0.28776726253066953
Root Mean Squared Error: 0.5364394304398862
Accuracy: 46.35605695601138
#### CONFUSION MATRIX ##########
[[0.05128205 0.07692308 0.08333333 0.05769231 0.51923077 0.05128205
  0.12820513 0.01923077 0.01282051]
 [0.0875     0.0125     0.125      0.0625     0.4875     0.0125
  0.1125     0.05       0.05      ]
 [0.0625     0.125      0.05208333 0.03125    0.5625     0.04166667
  0.01041667 0.01041667 0.10416667]
 [0.10204082 0.         0.06122449 0.06122449 0.51020408 0.02040816
  0.2244898  0.         0.02040816]
 [0.1047619  0.05714286 0